[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/06-YUV_compression/color-DCT_compression.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/contents/color_DCT/RD_performance.ipynb)

# Removing RGB redundancy with the [DCT](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.dct.html)

In [ ]:
%%bash
if [ -d "$HOME/repos" ]; then
    echo "\"$HOME/repos\" exists"
else
    mkdir ~/repos
    echo Created $HOME/repos
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/scalar_quantization" ]; then
    cd $HOME/repos/scalar_quantization
    echo "$HOME/repos/scalar_quantization ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/scalar_quantization.git
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/image_IO" ]; then
    cd $HOME/repos/image_IO
    echo "$HOME/repos/image_IO ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/image_IO.git
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/MRVC" ]; then
    cd $HOME/repos/MRVC
    echo "$HOME/repos/MRVC ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/Sistemas-Multimedia/MRVC.git
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/DCT" ]; then
    cd $HOME/repos/DCT
    echo "$HOME/repos/DCT ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/Sistemas-Multimedia/DCT.git
fi

In [ ]:
!ln -sf ~/repos/information_theory/information.py .
!ln -sf ~/repos/information_theory/distortion.py .
!ln -sf ~/repos/scalar_quantization/quantization.py .
!ln -sf ~/repos/scalar_quantization/deadzone_quantization.py .
!ln -sf ~/repos/image_IO/image_1.py .
!ln -sf ~/repos/image_IO/image_3.py .
!ln -sf ~/repos/image_IO/logging_config.py .
!ln -sf ~/repos/DCT/color_DCT.py .

In [ ]:
%matplotlib inline

#import matplotlib
#import matplotlib.pyplot as plt
#import matplotlib.axes as ax
#plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command

try:
    import matplotlib.pyplot as plt
except:
    !pip install matplotlib
    import matplotlib.pyplot as plt
%matplotlib inline

import pylab
import math
import numpy as np

try:
    from scipy import signal
except:
    !pip install scipy
    from scipy import signal
    
try:
    import cv2
except:
    !pip install opencv-python
    !pip install opencv-python-headless # Binder compatibility

try:
    import PIL
except:
    !pip install PIL
    
import os
import quantization
import deadzone_quantization as deadzone
#import midtread_quantizer as midtread
#import midrise_quantizer as midrise
import color_DCT
import distortion
import information
import image_3 as RGB_image
import image_1 as gray_image
import colored
#import common

## Configuration

In [ ]:
# Prefix of the RGB image to be compressed.

home = os.environ["HOME"]
fn = home + "/repos/MRVC/images/lena_color/"
#fn = home + "/MRVC/sequences/stockholm/"
image_dtype = np.uint8 # For 8 bpp/component images
#image_dtype = np.uint16 # For 16 bpp/component images

DCT_components = ['0', '1', '2']

# Number of quantization steps.
#N_Q_steps = 8
#Q_steps = [256*i/N_Q_steps for i in range(N_Q_steps + 1, 0, -1)]
Q_steps = [2**i for i in range(8, 0, -1)] # Simulate progressive bit-plane encoding.
#Q_steps = range(256, 32, -4)
print(Q_steps)

#quantizer = midtread
quantizer = deadzone.Deadzone_Quantizer
#quantizer = midrise

RGB_image.write = RGB_image.debug_write # Faster, but lower compression
#RGB_image.write = information.write # The fastest, but returns only an estimation of the length
gray_image.write = gray_image.debug_write # Faster, but lower compression
#gray_image.write = information.write # The fastest, but returns only an estimation of the length

## Read the image and show it

In [ ]:
RGB_img = RGB_image.read(fn).astype(image_dtype)
RGB_image.show(RGB_img, fn + "000.png")

In [ ]:
RGB_img.shape

In [ ]:
RGB_img.dtype

## (RGB -> color_DCT)-transform of the image

In [ ]:
import numpy as np
from scipy.fftpack import dct, idct

DCT_type = 3
norm = "ortho" # Orthonormal: orthogonal + unitary (unit gain in both directions of the transform)
#norm = None

In [ ]:
DCT_img = color_DCT.from_RGB(RGB_img)
print(DCT_img.dtype)

In [ ]:
gray_image.show_normalized(DCT_img[..., 0], fn + "000 (DCT0)")

In [ ]:
gray_image.show_normalized(DCT_img[..., 1], fn + "000 (DCT1)")

In [ ]:
gray_image.show_normalized(DCT_img[..., 2], fn + "000 (DCT2)")

## Energy of the DCT components

In [ ]:
energy_DCT0 = information.energy(DCT_img[..., 0])
energy_DCT1 = information.energy(DCT_img[..., 1])
energy_DCT2 = information.energy(DCT_img[..., 2])
print("energy_DCT0/energy_DCT1 =", energy_DCT0/energy_DCT1)
print("energy_DCT0/energy_DCT2 =", energy_DCT0/energy_DCT2)

As it can be seen:
1. Most of the energy is concentrated in the $\text{DCT}_0$ subband.
2. The coefficients can be positive and negative.
3. The dynamic range of the $\text{DCT}_0$ is more than twice the original one.

## Unitarity

In [ ]:
DCT0_avg_energy = information.average_energy(DCT_img[..., 0])
DCT1_avg_energy = information.average_energy(DCT_img[..., 1])
DCT2_avg_energy = information.average_energy(DCT_img[..., 2])
print(f"Average energy of component DCT0 = {int(DCT0_avg_energy)}")
print(f"Average energy of component DCT1 = {int(DCT1_avg_energy)}")
print(f"Average energy of component DCT2 = {int(DCT2_avg_energy)}")
total_DCT_avg_energy = DCT0_avg_energy + DCT1_avg_energy + DCT2_avg_energy
print(f"Total average energy (computed by adding the energies of the DCT coefficients {int(DCT0_avg_energy)} + {int(DCT1_avg_energy)} + {int(DCT2_avg_energy)}) = {int(total_DCT_avg_energy)}")
print(f"Total RGB average energy (computed directly from the RGB image) = {int(information.average_energy(RGB_img)*3)}")

Therefore, the forward DCT is energy preserving (unitary).

In [ ]:
RGB_recons_img = color_DCT.to_RGB(DCT_img)
print(f"Total RGB average energy (computed from the reconstructed RGB image) = {int(information.average_energy(RGB_recons_img)*3)}")

And the same can be said about the backward transform.

## (RGB <-> color-DCT) transform error

In [ ]:
RGB_image.show_normalized(RGB_recons_img, fn + "000.png (DCT recons)")

In [ ]:
np.array_equal(RGB_img, RGB_recons_img)

In [ ]:
print(RGB_img.max(), RGB_img.min())

In [ ]:
print(RGB_recons_img.max(), RGB_recons_img.min())

In [ ]:
RGB_image.show_normalized(RGB_img - RGB_recons_img.astype(image_dtype), "Reconstruction error (rounding error) color-DCT")

The DCT transform is irreversible. In general, only integer arithmetic operations guarantees reversibility.

## Relative gains of the synthesis filters

The synthesis filters gains are important because the quantization steps of each color-DCT component should be adjusted in order to effectively provide the desired number of [bins](http://www.winlab.rutgers.edu/~crose/322_html/quantization.pdf) (different dequantized values) in each component.

In [ ]:
def print_info(val):
    DCT0_delta = np.array([val, 0, 0])
    RGB_DCT0_delta = idct(DCT0_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT0_delta = information.energy(RGB_DCT0_delta)
    
    DCT1_delta = np.array([0, val, 0])
    RGB_DCT1_delta = idct(DCT1_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT1_delta = information.energy(RGB_DCT1_delta)
    
    DCT2_delta = np.array([0, 0, val])
    RGB_DCT2_delta = idct(DCT2_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT2_delta = information.energy(RGB_DCT2_delta)
    
    zero = np.array([0, 0, 0])
    RGB_zero = idct(zero, type=DCT_type, norm=norm)
    RGB_energy_zero = information.energy(RGB_zero)
    
    print(f"{val}^2 = {val*val}")
    
    print(f"Energy of {DCT0_delta} in the RGB domain ({RGB_DCT0_delta}) = {RGB_energy_DCT0_delta}")
    print(f"Energy of {DCT1_delta} in the RGB domain ({RGB_DCT1_delta}) = {RGB_energy_DCT1_delta}")
    print(f"Energy of {DCT2_delta} in the RGB domain ({RGB_DCT2_delta}) = {RGB_energy_DCT2_delta}")
    print(f"Energy of {zero} in the RGB domain ({RGB_zero}) = {RGB_energy_zero}")
    
    max_ = max(RGB_energy_DCT0_delta, RGB_energy_DCT1_delta, RGB_energy_DCT2_delta)
    DCT0_relative_gain = RGB_energy_DCT0_delta / max_
    DCT1_relative_gain = RGB_energy_DCT1_delta / max_
    DCT2_relative_gain = RGB_energy_DCT2_delta / max_
    print(f"Relative gain of DCT0 component = {DCT0_relative_gain}")
    print(f"Relative gain of DCT1 component = {DCT1_relative_gain}")
    print(f"Relative gain of DCT2 component = {DCT2_relative_gain}")
    
print_info(255)
print()
print_info(1)
print()
print_info(0)

The gain of each color-DCT inverse filter is 1. Therefore, under bit-rate restrictions, the optimal quantization pattern (without considering other aspects such as the compresibility of each component) is $\Delta_{\text{DCT0}} = \Delta_{\text{DCT1}} = \Delta_{\text{DCT2}}$.

## Amplitude shift in the DCT domain

To decide how to quantize, it is necessary to known how the amplitudes of the original image are *translated* to the transform domain. A good choice to find out this is to transform noise. In our case, lets use a random image with ([normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html)) [Gaussian noise](https://en.wikipedia.org/wiki/Gaussian_noise) with mean 0, color-DCT it, and check where the transformed noise has its mean in each component. Notice that, by definition, the noise cannot be decorrelated by transforms, and therefore, the noise is simply *transfered* from the RGB domain to the transform domain. Thus, depending on where the transformed noise has the mean, we can decide if the signal must be shifted before or after the transform. Notice that the input signal to a dead-zone quantizer must have 0 mean.

In [ ]:
# loc = mean, scale=standard deviation, size=number of samples
RGB_noise = np.random.normal(loc=0, scale=10, size=512*512*3).reshape(512, 512, 3)

In [ ]:
RGB_image.show(RGB_noise, "Gaussian RGB noise")

In [ ]:
DCT_noise = color_DCT.from_RGB(RGB_noise)

In [ ]:
gray_image.show_normalized(DCT_noise[..., 0], "Gaussian DCT0 noise")

In [ ]:
gray_image.show_normalized(DCT_noise[..., 1], "Gaussian DCT1 noise")

In [ ]:
gray_image.show_normalized(DCT_noise[..., 2], "Gaussian DCT2 noise")

The color-DCT does not modify the mean of the image (at least, when it has a 0 mean).

## RD curve using same Δ for each DCT subband ($\mathbf\Delta^{\text{DCT}^0}_i = \mathbf\Delta^{\text{DCT}^1}_i = \mathbf\Delta^{\text{DCT}^2}_i$)

In [ ]:
def DCT_same_delta_RD_curve_DCTdistortion(RGB_img, Q_steps, quantizer):
    RGB_img = RGB_img.astype(float)
    RGB_img[..., 0] -= np.average(RGB_img[..., 0])
    RGB_img[..., 1] -= np.average(RGB_img[..., 1])
    RGB_img[..., 2] -= np.average(RGB_img[..., 2])
    DCT_img = color_DCT.from_RGB(RGB_img)
    points = []
    for Q_step in Q_steps:
        print(f"q_step={Q_step}")
        Q = quantizer(Q_step=Q_step)
        DCT_y, k = Q.quan_dequan(DCT_img)
        #RGB_image.show_normalized(DCT_y)
        print(f"(Reconstructed components) max={np.max(DCT_y)} min={np.min(DCT_y)} avg={np.average(DCT_y)} dtype={DCT_y.dtype}")
        data_to_encode = (k + 128).astype(np.uint8)
        print(f"(Quantization indexes) max={np.max(data_to_encode)} min={np.min(data_to_encode)} avg={np.average(data_to_encode)} dtype={data_to_encode.dtype}")
        #BPP = common.bits_per_color_pixel(data_to_encode, str(Q_step) + '_')
        BPP = RGB_image.write(data_to_encode, str(Q_step) + '_', 0)*8/(data_to_encode.shape[0]*data_to_encode.shape[1])
        print("Used quantization indexes:", np.unique(data_to_encode))
        RMSE = distortion.RMSE(DCT_img, DCT_y)
        points.append((BPP, RMSE))
        print(f"rate={BPP} bits/pixel, distortion={RMSE}")
    return points

DCT_same_delta_RD_points_DCTdistortion = DCT_same_delta_RD_curve_DCTdistortion(RGB_img, Q_steps, quantizer)

In [ ]:
def DCT_same_delta_RD_curve_RGBdistortion(RGB_img, Q_steps, quantizer):
    RGB_img = RGB_img.astype(float)
    avg0 = np.average(RGB_img[..., 0])
    avg1 = np.average(RGB_img[..., 1])
    avg2 = np.average(RGB_img[..., 2])
    RGB_img[..., 0] -= avg0
    RGB_img[..., 1] -= avg1
    RGB_img[..., 2] -= avg2
    DCT_img = color_DCT.from_RGB(RGB_img)
    points = []
    for Q_step in Q_steps:
        Q = quantizer(Q_step=Q_step)
        DCT_y, k = Q.quan_dequan(DCT_img)
        data_to_encode = (k + 128).astype(np.uint8)
        #BPP = common.bits_per_color_pixel(data_to_encode, str(Q_step) + '_')
        BPP = RGB_image.write(data_to_encode, str(Q_step) + '_', 0)*8/(data_to_encode.shape[0]*data_to_encode.shape[1])
        print("Used quantization indexes:", np.unique(data_to_encode))
        RGB_y = color_DCT.to_RGB(DCT_y)
        RMSE = distortion.RMSE(RGB_img, RGB_y)
        #RGB_image.show_normalized(RGB_y, '')
        points.append((BPP, RMSE))
        print(f"q_step={Q_step}, rate={BPP} bits/pixel, distortion={RMSE}")
    return points

DCT_same_delta_RD_points_RGBdistortion = DCT_same_delta_RD_curve_RGBdistortion(RGB_img, Q_steps, quantizer)

In [ ]:
RGB_SQ = []
with open(f'../RGB_SQ/RGB_SQ.txt', 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_SQ.append((float(rate), float(_distortion)))

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_same_delta_RD_points_DCTdistortion), c='r', marker='.', label="Deadzone($\Delta_{\mathrm{DCT}^0} = \Delta_{\mathrm{DCT}^1} = \Delta_{\mathrm{DCT}^2}$) + PNG\n(distortion measured in DCT domain)", linestyle="dashed")
pylab.plot(*zip(*DCT_same_delta_RD_points_RGBdistortion), c='b', marker='.', label="Deadzone($\Delta_{\mathrm{DCT}^0} = \Delta_{\mathrm{DCT}^1} = \Delta_{\mathrm{DCT}^2}$) + PNG\n(distortion measured in RGB domain)", linestyle="dotted")
pylab.plot(*zip(*RGB_SQ), label="Deadzone($\mathbf{\Delta}^{\mathrm{R}}_i=\mathbf{\Delta}^{\mathrm{G}}_i=\mathbf{\Delta}^{\mathrm{B}}_i}$) + PNG")
pylab.title(fn)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
with open('RGB_DCT_SQ.txt', 'w') as f:
    for item in DCT_same_delta_RD_points_RGBdistortion:
        f.write(f"{item[0]}\t{item[1]}\n")

In [ ]:
import time
while True:
    time.sleep(1)

## Ignore the rest ...

## Compare to JPEG

In [ ]:
RGB_img = RGB_image.read(fn)
JPEG_RD_points = []
for q in range(75):
    im = PIL.Image.fromarray(RGB_img)
    im.save("/tmp/1.jpeg", quality=q, subsampling=0) # No color subsampling
    BPP = os.path.getsize("/tmp/1.jpeg")*8/(RGB_img.shape[0]*RGB_img.shape[1])
    zz = np.asarray(PIL.Image.open("/tmp/1.jpeg"))
    RMSE = distortion.RMSE(RGB_img, zz)
    JPEG_RD_points.append((BPP, RMSE))
    print(BPP, RMSE)

## Comparison with compressing (SQ+PNG) directly in the RGB domain

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RGB_points), c='b', marker='o', label="SQ+PNG in the RGB domain", linestyle="dashed")
pylab.plot(*zip(*DCT_same_delta_RD_points_DCTdistortion), c='r', marker='x', label="SQ+PNG in the DCT domain ($\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$)", linestyle="dotted")
pylab.title(fn)
pylab.xlabel("BPP")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

The color-DCT removes the color redundancy, generating three components that are more compressible than the original RGB channels. This effect is more significant a low bit-rates. 

## RDO (Rate/Distortion Optimization)
RDO tries to find a quantization steps pattern (in our case, a combination of 3 quantization steps, one per component) that minimizes the distortion given a predefined bit budget. In general, such pattern can be determined without considering any extra restriction, such as that the quantization steps patterns can be progressive, or what is the same, simulating a bit-plane encoding (in our case, of the color-DCT components).

In general, RDO is a [NP-hard](https://en.wikipedia.org/wiki/NP-hardness) [combinatorial problem](/https://en.wikipedia.org/wiki/Combinatorial_optimization), and to find an optimal solution we must use [exhautive search](https://en.wikipedia.org/wiki/Brute-force_search) tecnhiques.

## Progressive bit-plane encoding using RDO
When the components must be "transmitted" by bitplanes, the "search area" in which to find an optimal solution is much smaller, because the search tree has only (in the case of the color-DCT) 3 branches at each node. For example, if the first quantization step pattern is `[256, 256, 256]` (which considering 8 bit-planes will "send" the MSb), we need to decide between `[128, 256, 256]`, `[256, 128, 256]`, or `[256, 256, 128]`, determining which pattern has the highest RD slope. Notice, however, that the resulting RD curve not necesarily will be optimal, because this is only achieved when the RD slope of each component is the same.

## RDO over orthogonal spaces
When the components to encode are orthogonal (as it is the case of the color-DCT domain), their contributions to the quality of the image are independent and additive, and therefore the distortion can be measured in the original signal domain and the transformed one. If it is also true that if the encoding performance of the components is similar, the constant quantization pattern $\Delta_{\text{DCT0}} = \Delta_{\text{DCT1}} = \Delta_{\text{DCT2}}$ should provide, in general, a good RDO solution because the rate will be proportional to the energy of the components, generating RD points that satisfy the constant-slope requirement.

## A fast RDO algorithm for color-DCTed images
This RDO algorithm takes advantage of the orthogonality of the color-DCT:
1. Compute the RD curve of each component, measuring the distortion in the component domain.
2. Compute the slope of the points in ther curves.
3. Remove those points that does not belong to the convex-hull of the RD curve.
4. Sort all the points by their slope in their curves.
5. Use such ordering for finding the progression of quantization steps patters.

### 1. RD curve of each color-DCT component

In [ ]:
def color_DCT_RD_curve_per_component(RGB_img, Q_steps, Q, components):
    N_components = len(components)
    RGB_img = RGB_img.astype(float)
    for c in range(N_components):
        avg = np.average(RGB_img[..., c])
        RGB_img[..., c] -= avg
        print(f"channel={c} average={avg}")
    DCT_img = color_DCT.from_RGB(RGB_img)
    DCT_img_copy = DCT_img.copy()
    RD_points = []
    for c in range(N_components):
        #RD_points.append([])
        RD_points.append([(0, 100000, c, 512)])
    for Q_step in Q_steps:
        Q = quantizer(Q_step=Q_step)
        DCT_k = Q.quantize(DCT_img)
        for component_index in range(N_components):
            component_name = components[component_index]
            DCT_y = np.zeros_like(DCT_img)
            DCT_y = DCT_img_copy.copy()
            DCT_y[..., component_index] = Q.dequantize(DCT_k[..., component_index])
            #print(Q_step, DCT_k[..., component_index].max(), DCT_k[..., component_index].min(), information.entropy(DCT_k[..., component_index]))
            #BPP = common.bits_per_gray_pixel(DCT_k[..., component_index] + 128, str(Q_step) + '_' + component_name + '_' + str(components[component_index]))
            BPP = gray_image.write(DCT_k[..., component_index] + 128, str(Q_step) + '_' + component_name + '_' + str(components[component_index]), 0)*8/DCT_k[..., component_index].size
            print("Used quantization indexes:", np.unique(DCT_k[..., component_index] + 128))
            #RGB_y = color_DCT.to_RGB(DCT_y)
            #_distortion = distortion.RMSE(RGB_img, RGB_y)
            RMSE = distortion.RMSE(DCT_img[..., component_index], DCT_y[..., component_index])
            #common.show(RGB_y, components[component_index] + ' ' + str(Q_step))
            RD_points[component_index].append((BPP, RMSE, component_name, Q_step))
            print(f"component_index={components[component_index]} q_step={Q_step}, rate={BPP} bits/pixel, distortion={RMSE}")
    return RD_points
DCT_RD_curve_per_component = color_DCT_RD_curve_per_component(RGB_img, Q_steps, quantizer, DCT_components)

In [ ]:
DCT_RD_curve_per_component

### Display the curve of each component

In [ ]:
pylab.figure(dpi=150)
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[0]]), c='r', marker='.', label="$\mathrm{DCT0}}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[1]]), c='g', marker='.', label="$\mathrm{DCT1}}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[2]]), c='b', marker='.', label="$\mathrm{DCT2}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[0][1:]]), c='r', marker='.', label="$\mathrm{DCT0}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[1][1:]]), c='g', marker='.', label="$\mathrm{DCT1}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[2][1:]]), c='b', marker='.', label="$\mathrm{DCT2}}$", linestyle="dashed")
pylab.title(fn)
pylab.xlabel("BPP")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

### 2. Compute the slope of each point

In [ ]:
def compute_slopes(RD_points):
    counter = 0
    RD_slopes = []
    points_iterator = iter(RD_points)
    next(points_iterator)
    for i in points_iterator:
        BPP = i[0] # Rate 
        delta_BPP = BPP - RD_points[counter][0]
        RMSE = i[1] # Distortion
        delta_RMSE = RMSE - RD_points[counter][1] 
        if delta_BPP > 0:
            slope = abs(delta_RMSE/delta_BPP)
        else:
            slope = 0
        component = i[2]
        Q_step = i[3]
        print((slope, i), delta_RMSE, delta_BPP)
        RD_slopes.append((slope, component, Q_step))
        counter += 1
    return RD_slopes

In [ ]:
#DCT0_slopes = common.compute_slopes(DCT_RD_curve_per_component[0])
#DCT1_slopes = common.compute_slopes(DCT_RD_curve_per_component[1])
#DCT2_slopes = common.compute_slopes(DCT_RD_curve_per_component[2])
DCT0_slopes = compute_slopes(DCT_RD_curve_per_component[0])
print()
DCT1_slopes = compute_slopes(DCT_RD_curve_per_component[1])
print()
DCT2_slopes = compute_slopes(DCT_RD_curve_per_component[2])

In [ ]:
DCT0_slopes

In [ ]:
DCT1_slopes

In [ ]:
DCT2_slopes

### 3. Filter the curves
Remove those RD points that do not belong to the [convex-hull](https://en.wikipedia.org/wiki/Convex_hull).

In [ ]:
def filter_slopes(slopes):
    filtered_slopes = []
    slopes_iterator = iter(slopes)
    prev = next(slopes_iterator)
    for curr in slopes_iterator:
        if prev[0] < curr[0]:
            print(f"deleted {prev}")
        else:
            filtered_slopes.append(prev)
        prev = curr
    filtered_slopes.append(prev)
    return filtered_slopes


In [ ]:
DCT0_slopes = filter_slopes(DCT0_slopes)
DCT0_slopes

In [ ]:
DCT1_slopes = filter_slopes(DCT1_slopes)
DCT1_slopes

In [ ]:
DCT2_slopes = filter_slopes(DCT2_slopes)
DCT2_slopes

### 4. Sort the slopes at each quantization step
Notice that the TPs (Truncation Points) generated in a component must be used in order (we cannot use the Q_step 32 after using 16, for example).

In [ ]:
all_slopes = DCT0_slopes + DCT1_slopes + DCT2_slopes
sorted_slopes = sorted(all_slopes, key=lambda x: x[0])[::-1]
sorted_slopes

### 5. Compute the optimal RD curve
And finally, let's compute the RD curve (remember that the previous points are only an estimation of the order in which the quantization steps should be increased in each component to build the RD curve, not the real RD curve that measures the distortion in the RGB domain).

In [ ]:
def DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, components):
    RGB_img = RGB_img.astype(float)
    RGB_img[..., 0] -= np.average(RGB_img[..., 0])
    RGB_img[..., 1] -= np.average(RGB_img[..., 1])
    RGB_img[..., 2] -= np.average(RGB_img[..., 2])
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image (the average of each component is 0).
    DCT_img = color_DCT.from_RGB(RGB_img)
    k = np.empty_like(DCT_img)
    DCT_y = np.empty_like(DCT_img)
    for i in sorted_slopes:
        print(i)
        component, Q_step = i[1], i[2]
        Q_steps_per_component[components.index(component)] = Q_step
        for c, Q_step in zip(components, Q_steps_per_component):
            component_index = components.index(c)
            Q = quantizer(Q_step=Q_step)
            DCT_y[..., component_index], k[..., component_index] = Q.quan_dequan(DCT_img[..., component_index])
        #rate = common.bits_per_color_pixel(k.astype(np.uint8), str(Q_steps_per_component) + '_')
        rate = RGB_image.write(k.astype(np.uint8), str(Q_steps_per_component).replace(" ", "").replace("[", "").replace("]","").replace(",","_") + '_', 0)*8/(k.shape[0]*k.shape[1])
        #y_RGB = YCrCb.to_RGB(y + 128)
        #RGB_y = color_DCT.to_RGB(DCT_y)               # Uncomment to compute distortion
        #_distortion = distortion.RMSE(RGB_img, RGB_y)  # in the RGB domain.
        _distortion = distortion.RMSE(DCT_img, DCT_y)  # Uncomment to compute distortion in the DCT domain.
        #RGB_image.show(y_RGB, f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

def quantize(DCT_img, DCT_y, k, components, Q_steps):
    for c, Q_step in zip(components, Q_steps):
        component_index = components.index(c)
        Q = quantizer(Q_step=Q_step)
        DCT_y[..., component_index], k[..., component_index] = Q.quan_dequan(DCT_img[..., component_index])
    return DCT_y

def compute_RD_point(DCT_img, DCT_y, k, components, Q_steps):
    DCT_y = quantize(DCT_img, DCT_y, k, components, Q_steps)
    rate = RGB_image.write(k.astype(np.uint8), str(Q_steps).replace(" ", "").replace("[", "").replace("]","").replace(",","_") + '_', 0)*8/(k.shape[0]*k.shape[1])
    _distortion = distortion.RMSE(DCT_img, DCT_y)
    return (rate, _distortion)
    
def compute_slope(prev_RD_point, next_RD_point):
    delta_rate = next_RD_point[0] - prev_RD_point[0]
    delta_distortion = prev_RD_point[1] - next_RD_point[1] 
    if delta_rate > 0:
        slope = abs(delta_distortion/delta_rate)
    else:
        slope = 0
    return slope
 
def get_index_of_max(inputlist):
    min_value = max(inputlist)
    index_of_min = inputlist.index(min_value)
    return index_of_min

def DCT_optimal_curve2(RGB_img, quantizer, components):
    RGB_img = RGB_img.astype(float)
    RGB_img[..., 0] -= np.average(RGB_img[..., 0])
    RGB_img[..., 1] -= np.average(RGB_img[..., 1])
    RGB_img[..., 2] -= np.average(RGB_img[..., 2])
    points = []
    Q_steps = [128, 128, 128]
    DCT_img = color_DCT.from_RGB(RGB_img)
    k = np.empty_like(DCT_img)
    DCT_y = np.empty_like(DCT_img)
    RD_point = compute_RD_point(DCT_img, DCT_y, k, components, Q_steps)
    slopes = [0, 0, 0]
    while Q_steps != [1, 1, 1]:
        points.append(RD_point)
        print(RD_point, Q_steps)
        A = compute_RD_point(DCT_img, DCT_y, k, components, [(Q_steps[0]+1)//2, Q_steps[1], Q_steps[2]])
        B = compute_RD_point(DCT_img, DCT_y, k, components, [Q_steps[0], (Q_steps[1]+1)//2, Q_steps[2]])
        C = compute_RD_point(DCT_img, DCT_y, k, components, [Q_steps[0], Q_steps[1], (Q_steps[2]+1)//2])
        slopes[0] = compute_slope(RD_point, A)
        slopes[1] = compute_slope(RD_point, B)
        slopes[2] = compute_slope(RD_point, C)
        max_slope_index = get_index_of_max(slopes)
        if max_slope_index == 0:
            Q_steps = [(Q_steps[0]+1)//2, Q_steps[1], Q_steps[2]]
            RD_point = A
            points.append(A)
        elif max_slope_index == 1:
            Q_steps = [Q_steps[0], (Q_steps[1]+1)//2, Q_steps[2]]
            RD_point = B
            points.append(B)
        else:
            Q_steps = [Q_steps[0], Q_steps[1], (Q_steps[2]+1)//2]
            RD_point = C
            points.append(C)
    return points

#DCT_optimal_RD_points = DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, DCT_components)
DCT_optimal_RD_points = DCT_optimal_curve2(RGB_img, quantizer, DCT_components)
DCT_optimal_RD_points

## Compare curves

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_same_delta_RD_points_DCTdistortion), c='b', marker='o', label="$\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$", linestyle="dashed")
pylab.plot(*zip(*DCT_optimal_RD_points), c='r', marker='x', label="RDO", linestyle="dotted")
pylab.title(fn)
pylab.xlabel("BPP")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

## Save

## Compare to VQ

In [ ]:
def load(path):
    curve = []
    with open(path, 'r') as f:
        for line in f:
            rate, _distortion = line.split('\t')
            curve.append((float(rate), float(_distortion)))
    return curve

In [ ]:
VQ = load("../RGB_VQ/RGB_VQ.txt")

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_optimal_RD_points), c='m', marker='x', label="DCT", linestyle="dotted")
pylab.plot(*zip(*VQ), c='k', marker='x', label="VQ", linestyle="dotted")
pylab.title(fn)
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
#pylab.xscale("log")
#pylab.yscale("log")
pylab.show()

## Conclusions

1. The (color) DCT is orthogonal. The distortion can be computed in the DCT domain.
2. The (color) DCT is unitary. The same quantization step sizes can be used for the DCT subbands.
3. The performances of the RDO algorithm and the constant quantization algorithm are very similar. This happens because the constant quantization algorithm achieves to use approximately the same slope for the three components in each RD point.

In [ ]:
import time
while True:
    time.sleep(1)

## Another solutions ... (ignore)

In [ ]:
def filter_points(points):
    filtered_points = []
    points_iterator = iter(points)
    prev = next(points_iterator)
    for curr in points_iterator:
        if prev[0] >= curr[0]:
            print(f"deleted {prev}")
        else:
            filtered_points.append(prev)
        prev = curr
    filtered_points.append(prev)
    return filtered_points

for c in range(3):
    DCT_RD_curve_per_component[c] = filter_points(DCT_RD_curve_per_component[c])

In [ ]:
DCT_RD_curve_per_component

## Display the curve of each component

In [ ]:
pylab.figure(dpi=150)
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[0]]), c='r', marker='.', label="$\mathrm{DCT0}}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[1]]), c='g', marker='.', label="$\mathrm{DCT1}}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[2]]), c='b', marker='.', label="$\mathrm{DCT2}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[0][1:]]), c='r', marker='.', label="$\mathrm{DCT0}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[1][1:]]), c='g', marker='.', label="$\mathrm{DCT1}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[2][1:]]), c='b', marker='.', label="$\mathrm{DCT2}}$", linestyle="dashed")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE in RGB domain")
pylab.legend(loc='upper right')
pylab.show()

Each curve shows the impact of variying the quantization step, in the corresponding component. Moreover, since the color-DCT is orthonormal, the gain of each component is 1. Therefore, the relative position of the curves shows the contribution (remember that if we are using the RMSE as the distortion metric and for this reason, me are basically measuring energy) of each component to the quality of the reconstructed image. As an example, we could use the following progression of quantization steps patterns:
1. `[256, 256, 256]` -> Constant gray image.
2. `[128, 256, 256]` -> Grayscale image.
3. `[ 64, 256, 256]` -> A higher quality grayscale image.
4. `[ 64, 128, 256]` -> Add some color.
5. `[ 64, 128, 128]` -> Add more color.
6. And so on ...

In [ ]:
def color_DCT_optimal_curve(RGB_img, RD_points_per_component):
    RGB_img = RGB_img.astype(float)
    RGB_img[..., 0] -= np.average(RGB_img[..., 0])
    RGB_img[..., 1] -= np.average(RGB_img[..., 1])
    RGB_img[..., 2] -= np.average(RGB_img[..., 2])
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image (the average of each component is 0).
    DCT_img = color_DCT.from_RGB(RGB_img)
    k = np.empty_like(DCT_img)
    DCT_y = np.empty_like(DCT_img)
    gains = [RD_points_per_component[0][0][1],
             RD_points_per_component[1][0][1],
             RD_points_per_component[2][0][1]]
    max_gain = max(gains)
    where = np.argmax(gains)
    Q_steps_indexes = [0, 0, 0]
    prev_Q_steps_indexes = [0, 0, 0]
    #Q_steps_indexes[where] += 1
    while Q_steps_per_component != [4, 4, 4]:
        Q_steps_indexes[where] += 1
        Q_steps_per_component = [256, 256, 256]
        for i in range(3):
            Q_steps_per_component[i] >>= Q_steps_indexes[i]
        print(f"{Q_steps_indexes} Quantization steps pattern: {Q_steps_per_component}")
        for component_index, Q_step in zip([0, 1, 2], Q_steps_per_component):
            DCT_y[..., component_index], k[..., component_index] = quantizer.quan_dequan(DCT_img[..., component_index], Q_step)
        BPP = common.bits_per_color_pixel((k + 128).astype(np.uint8), str(Q_steps_per_component) + '_')
        print("Used quantization indexes:", np.unique((k + 128).astype(np.uint8)))
        RMSE = distortion.RMSE(DCT_img, DCT_y)
        points.append((BPP, RMSE))
        print(f"rate={BPP} bits/pixel, distortion={RMSE}")
        slopes = []
        print("Considering")
        for c in range(3):
            if len(RD_points_per_component[c]) > (Q_steps_indexes[c] + 1):
                delta_BPP = RD_points_per_component[c][Q_steps_indexes[c] + 1][0] - RD_points_per_component[c][Q_steps_indexes[c]][0]
                delta_RMSE = RD_points_per_component[c][Q_steps_indexes[c]][1] - RD_points_per_component[c][Q_steps_indexes[c] + 1][1]            #print(prev_Q_steps_indexes, Q_steps_indexes, RD_points_per_component[c][prev_Q_steps_indexes[0]], RD_points_per_component[c][Q_steps_indexes[0]])
                #print(f"delta_BPP={delta_BPP}")
                if delta_BPP > 0:
                    slope = delta_RMSE/delta_BPP
                else:
                    slope = 0
                slopes.append(slope)
            else:
                slopes.append(0)
        print(f"slopes={slopes}")
        where = np.argmax(slopes)
        print(f"where={where}")
        prev_Q_steps_indexes = Q_steps_indexes.copy()
    return points

DCT_optimal_RD_points = color_DCT_optimal_curve(RGB_img, DCT_RD_curve_per_component)
DCT_optimal_RD_points

## Compare the curves

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_same_delta_RD_points_DCTdistortion), c='b', marker='o', label="$\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$", linestyle="dashed")
pylab.plot(*zip(*DCT_optimal_RD_points), c='r', marker='x', label="RDO", linestyle="dotted")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

## Compute the slope of each point

In [ ]:
def compute_slopes(RD_points):
    counter = 0
    RD_slopes = []
    points_iterator = iter(RD_points)
    next(points_iterator)
    for i in points_iterator:
        BPP = i[0] # Rate 
        delta_BPP = BPP - RD_points[counter][0]
        RMSE = i[1] # Distortion
        delta_RMSE = RMSE - RD_points[counter][1] 
        if delta_BPP > 0:
            slope = abs(delta_RMSE/delta_BPP)
        else:
            slope = 0
        component = i[2]
        Q_step = i[3]
        print((slope, i), delta_RMSE, delta_BPP)
        RD_slopes.append((slope, component, Q_step))
        counter += 1
    return RD_slopes

In [ ]:
#DCT0_slopes = common.compute_slopes(DCT_RD_curve_per_component[0])
#DCT1_slopes = common.compute_slopes(DCT_RD_curve_per_component[1])
#DCT2_slopes = common.compute_slopes(DCT_RD_curve_per_component[2])
DCT0_slopes = compute_slopes(DCT_RD_curve_per_component[0])
DCT1_slopes = compute_slopes(DCT_RD_curve_per_component[1])
DCT2_slopes = compute_slopes(DCT_RD_curve_per_component[2])

In [ ]:
DCT0_slopes

In [ ]:
DCT1_slopes

In [ ]:
DCT2_slopes

## Filter the curves
Remove those RD points that do not belong to the convex-hull.

In [ ]:
DCT0_slopes = common.filter_slopes(DCT0_slopes)
DCT0_slopes

In [ ]:
DCT1_slopes = common.filter_slopes(DCT1_slopes)
DCT1_slopes

In [ ]:
DCT2_slopes = common.filter_slopes(DCT2_slopes)
DCT2_slopes = common.filter_slopes(DCT2_slopes)
DCT2_slopes

## Sort the slopes at each quantization step
Notice that the TPs (Truncation Points) generated in a component must be used in order.

In [ ]:
all_slopes = DCT0_slopes + DCT1_slopes + DCT2_slopes
sorted_slopes = sorted(all_slopes, key=lambda x: x[0])[::-1]
sorted_slopes

## Compute the optimal RD curve
And finally, let's compute the RD curve (remember that the previous points are only an estimation of the order in which the quantization steps should be increased in each component to build the RD curve, not the real RD curve that measures the distortion in the RGB domain).

In [ ]:
def DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, components):
    RGB_img = RGB_img.astype(float)
    RGB_img[..., 0] -= np.average(RGB_img[..., 0])
    RGB_img[..., 1] -= np.average(RGB_img[..., 1])
    RGB_img[..., 2] -= np.average(RGB_img[..., 2])
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image (the average of each component is 0).
    DCT_img = color_DCT.from_RGB(RGB_img)
    k = np.empty_like(DCT_img)
    DCT_y = np.empty_like(DCT_img)
    for i in sorted_slopes:
        print(i)
        component, Q_step = i[1], i[2]
        Q_steps_per_component[components.index(component)] = Q_step
        for c, Q_step in zip(components, Q_steps_per_component):
            component_index = components.index(c)
            DCT_y[..., component_index], k[..., component_index] = quantizer.quan_dequan(DCT_img[..., component_index], Q_step)
        rate = common.bits_per_color_pixel((k + 128).astype(np.uint8), str(Q_steps_per_component) + '_')
        print("Used quantization indexes:", np.unique((k + 128).astype(np.uint8)))
        #y_RGB = YCrCb.to_RGB(y + 128)
        #RGB_y = color_DCT.to_RGB(DCT_y)               # Uncomment to compute distortion
        #_distortion = distortion.RMSE(RGB_img, RGB_y)  # in the RGB domain.
        _distortion = distortion.RMSE(DCT_img, DCT_y)  # Uncomment to compute distortion in the DCT domain.
        #common.show(y_RGB, f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points



DCT_optimal_RD_points = DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, DCT_components)
DCT_optimal_RD_points

## Compare the curves

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_same_delta_RD_points_RGBdistortion), c='b', marker='o', label="$\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$", linestyle="dashed")
pylab.plot(*zip(*DCT_optimal_RD_points), c='r', marker='x', label="Optimal", linestyle="dotted")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
with open('color_DCT.txt', 'w') as f:
    for item in DCT_optimal_RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

## Conclusión
The performance of the simple quantization solution ($\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$) is very close to the optimal one. This is normal because the DCT is orthogonal.

In [ ]:
def color_DCT_optimal_curve(RGB_img, RD_points_per_component):
    RGB_img = RGB_img.astype(float)
    RGB_img[..., 0] -= np.average(RGB_img[..., 0])
    RGB_img[..., 1] -= np.average(RGB_img[..., 1])
    RGB_img[..., 2] -= np.average(RGB_img[..., 2])
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image (the average of each component is 0).
    DCT_img = color_DCT.from_RGB(RGB_img)
    k = np.empty_like(DCT_img)
    DCT_y = np.empty_like(DCT_img)
    max_gain = np.max(RD_points_per_component[0])
    while Q_steps_per_component != [2, 2, 2]:
        
    for i in sorted_slopes:
        print(i)
        component, Q_step = i[1], i[2]
        Q_steps_per_component[components.index(component)] = Q_step
        for c, Q_step in zip(components, Q_steps_per_component):
            component_index = components.index(c)
            DCT_y[..., component_index], k[..., component_index] = quantizer.quan_dequan(DCT_img[..., component_index], Q_step)
        rate = common.bits_per_color_pixel((k + 128).astype(np.uint8), str(Q_steps_per_component) + '_')
        print("Used quantization indexes:", np.unique((k + 128).astype(np.uint8)))
        #y_RGB = YCrCb.to_RGB(y + 128)
        #RGB_y = color_DCT.to_RGB(DCT_y)               # Uncomment to compute distortion
        #_distortion = distortion.RMSE(RGB_img, RGB_y)  # in the RGB domain.
        _distortion = distortion.RMSE(DCT_img, DCT_y)  # Uncomment to compute distortion in the DCT domain.
        #common.show(y_RGB, f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

DCT_optimal_RD_points = DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, DCT_components)
DCT_optimal_RD_points

## Optimal RD curve (measuring distortion in the RGB domain)
Useful when the transform is not orthogonal.

## RD curve of each DCT component

Notice that the distortion has been measured in the RGB domain and that the components are not 

In [ ]:
def color_DCT_RD_curve_per_component_(RGB_img, Q_steps, Q, components):
    N_components = len(components)
    RGB_img = RGB_img.astype(float)
    for c in range(N_components):
        avg = np.average(RGB_img[..., c])
        RGB_img[..., c] -= avg
        print(f"channel={c} average={avg}")
    DCT_img = color_DCT.from_RGB(RGB_img)
    DCT_img_copy = DCT_img.copy()
    RD_points = []
    for c in range(N_components):
        RD_points.append([])
    for Q_step in Q_steps:
        DCT_k = Q.quantize(DCT_img, Q_step)
        for component_index in range(N_components):
            component_name = components[component_index]
            DCT_y = np.zeros_like(DCT_img)
            DCT_y = DCT_img_copy.copy()
            DCT_y[..., component_index] = Q.dequantize(DCT_k[..., component_index], Q_step)
            #print(Q_step, DCT_k[..., component_index].max(), DCT_k[..., component_index].min(), information.entropy(DCT_k[..., component_index]))
            rate = common.bits_per_gray_pixel(DCT_k[..., component_index], str(Q_step) + '_' + component_name + '_' + str(components[component_index]))
            RGB_y = color_DCT.to_RGB(DCT_y)
            _distortion = distortion.RMSE(RGB_img, RGB_y)
            #common.show(RGB_y, components[component_index] + ' ' + str(Q_step))
            RD_points[component_index].append((rate, _distortion, component_name, Q_step))
            print(f"component_index={components[component_index]} q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return RD_points
           
def color_DCT_RD_curve_per_component(RGB_img, Q_steps, Q, components):
    N_components = len(components)
    RGB_img = RGB_img.astype(float)
    for c in range(N_components):
        avg = np.average(RGB_img[..., c])
        RGB_img[..., c] -= avg
        print(f"channel={c} average={avg}")
    DCT_img = color_DCT.from_RGB(RGB_img)
    DCT_img_copy = DCT_img.copy()
    RD_points = []
    for c in range(N_components):
        RD_points.append([(0, 100000, c, 512)])
        #RD_points.append([])
    for Q_step in Q_steps:
        for component_index in range(N_components):
            component_name = components[component_index]
            #DCT_k = np.zeros_like(DCT_img)
            DCT_k = DCT_img_copy.copy()
            DCT_y = DCT_img_copy.copy()
            DCT_k[..., component_index] = Q.quantize(DCT_img[..., component_index], Q_step)
            DCT_y[..., component_index] = Q.dequantize(DCT_k[..., component_index], Q_step)
            #print(Q_step, DCT_k[..., component_index].max(), DCT_k[..., component_index].min(), information.entropy(DCT_k[..., component_index]))
            BPP = common.bits_per_gray_pixel(DCT_k[..., component_index].astype(np.uint8), str(Q_step) + '_' + component_name + '_' + str(components[component_index]))
            RGB_y = color_DCT.to_RGB(DCT_y)
            RMSE = distortion.RMSE(RGB_img, RGB_y)
            #common.show(RGB_y, components[component_index] + ' ' + str(Q_step))
            RD_points[component_index].append((BPP, RMSE, component_name, Q_step))
            print(f"component_index={components[component_index]} q_step={Q_step}, rate={BPP} bits/pixel, distortion={RMSE}")
    return RD_points

DCT_RD_curve_per_component = color_DCT_RD_curve_per_component(RGB_img, Q_steps, quantizer, DCT_components)

In [ ]:
DCT_RD_curve_per_component

## Display the curve of each component

In [ ]:
pylab.figure(dpi=150)
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[0]]), c='r', marker='.', label="$\mathrm{DCT0}}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[1]]), c='g', marker='.', label="$\mathrm{DCT1}}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[2]]), c='b', marker='.', label="$\mathrm{DCT2}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[0][1:]]), c='r', marker='.', label="$\mathrm{DCT0}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[1][1:]]), c='g', marker='.', label="$\mathrm{DCT1}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_RD_curve_per_component[2][1:]]), c='b', marker='.', label="$\mathrm{DCT2}}$", linestyle="dashed")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE in RGB domain")
pylab.legend(loc='upper right')
pylab.show()

Notice that this plot represents the impact in the RGB domain of quantizing each component, keeping the rest unquantized. Therefore, the most important component from a RD point of view is DCT0.

## Compute the slope of each point

In [ ]:
def compute_slopes(RD_points):
    counter = 0
    RD_slopes = []
    points_iterator = iter(RD_points)
    next(points_iterator)
    for i in points_iterator:
        BPP = i[0] # Rate 
        delta_BPP = BPP - RD_points[counter][0]
        RMSE = i[1] # Distortion
        delta_RMSE = RMSE - RD_points[counter][1] 
        if delta_BPP > 0:
            slope = abs(delta_RMSE/delta_BPP)
        else:
            slope = 0
        component = i[2]
        Q_step = i[3]
        print((slope, i), delta_RMSE, delta_BPP)
        RD_slopes.append((slope, component, Q_step))
        counter += 1
    return RD_slopes

In [ ]:
#DCT0_slopes = common.compute_slopes(DCT_RD_curve_per_component[0])
#DCT1_slopes = common.compute_slopes(DCT_RD_curve_per_component[1])
#DCT2_slopes = common.compute_slopes(DCT_RD_curve_per_component[2])
DCT0_slopes = compute_slopes(DCT_RD_curve_per_component[0])
DCT1_slopes = compute_slopes(DCT_RD_curve_per_component[1])
DCT2_slopes = compute_slopes(DCT_RD_curve_per_component[2])

In [ ]:
DCT0_slopes

In [ ]:
DCT1_slopes

In [ ]:
DCT2_slopes

## Filter the curves
Remove those RD points that do not belong to the convex-hull.

In [ ]:
DCT0_slopes = common.filter_slopes(DCT0_slopes)
DCT0_slopes

In [ ]:
DCT1_slopes = common.filter_slopes(DCT1_slopes)
DCT1_slopes

In [ ]:
DCT2_slopes = common.filter_slopes(DCT2_slopes)
DCT2_slopes = common.filter_slopes(DCT2_slopes)
DCT2_slopes

## Sort the slopes at each quantization step
Notice that the TPs (Truncation Points) generated in a component must be used in order.

In [ ]:
all_slopes = DCT0_slopes + DCT1_slopes + DCT2_slopes
sorted_slopes = sorted(all_slopes, key=lambda x: x[0])[::-1]
sorted_slopes

## Compute the optimal RD curve
And finally, let's compute the RD curve (remember that the previous points are only an estimation of the order in which the quantization steps should be increased in each component to build the RD curve, not the real RD curve that measures the distortion in the RGB domain).

In [ ]:
def DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, components):
    RGB_img = RGB_img.astype(float)
    RGB_img[..., 0] -= np.average(RGB_img[..., 0])
    RGB_img[..., 1] -= np.average(RGB_img[..., 1])
    RGB_img[..., 2] -= np.average(RGB_img[..., 2])
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image (the average of each component is 0).
    DCT_img = color_DCT.from_RGB(RGB_img)
    k = np.empty_like(DCT_img)
    DCT_y = np.empty_like(DCT_img)
    for i in sorted_slopes:
        print(i)
        component, Q_step = i[1], i[2]
        Q_steps_per_component[components.index(component)] = Q_step
        for c, Q_step in zip(components, Q_steps_per_component):
            component_index = components.index(c)
            DCT_y[..., component_index], k[..., component_index] = quantizer.quan_dequan(DCT_img[..., component_index], Q_step)
        rate = common.bits_per_color_pixel(k.astype(np.uint8), str(Q_steps_per_component) + '_')
        #y_RGB = YCrCb.to_RGB(y + 128)
        #RGB_y = color_DCT.to_RGB(DCT_y)               # Uncomment to compute distortion
        #_distortion = distortion.RMSE(RGB_img, RGB_y)  # in the RGB domain.
        _distortion = distortion.RMSE(DCT_img, DCT_y)  # Uncomment to compute distortion in the DCT domain.
        #common.show(y_RGB, f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

DCT_optimal_RD_points = DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, DCT_components)
DCT_optimal_RD_points

## Compare the curves

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_same_delta_RD_points_RGBdistortion), c='b', marker='o', label="$\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$", linestyle="dashed")
pylab.plot(*zip(*DCT_optimal_RD_points), c='r', marker='x', label="Optimal", linestyle="dotted")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
with open('color_DCT.txt', 'w') as f:
    for item in DCT_optimal_RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

## Conclusión
The performance of the simple quantization solution ($\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$) is very close to the optimal one. This is normal because the DCT is orthogonal.

## Compare to quantizing directly in the RGB domain

In [ ]:
RGB_points = []
with open(f'../05-RGB_compression/RGB.txt', 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_points.append((float(rate), float(_distortion)))

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RGB_points), c='b', marker='o', label="Quantization in the RGB domain", linestyle="dashed")
pylab.plot(*zip(*DCT_optimal_RD_points), c='r', marker='x', label="Quantization in the color-DCT domain", linestyle="dotted")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()